![A car dashboard with lots of new technical features.](images/dashboard.jpg)

You're working for a well-known car manufacturer who is looking at implementing LLMs into vehicles to provide guidance to drivers. You've been asked to experiment with integrating car manuals with an LLM to create a context-aware chatbot. They hope that this context-aware LLM can be hooked up to a text-to-speech software to read the model's response aloud.

As a proof of concept, you'll integrate several pages from a car manual that contains car warning messages and their meanings and recommended actions. This particular manual, stored as an HTML file, `mg-zs-warning-messages.html`, is from an MG ZS automobile, a compact SUV. Armed with your newfound knowledge of LLMs and LangChain, you'll implement Retrieval Augmented Generation (RAG) to create the context-aware chatbot.

**Note: Although we'll be using the OpenAI API in this project, you do not need to specify an API key.**

In [15]:
# Run this cell to install the necessary packages
import subprocess
import pkg_resources

def install_if_needed(package, version):
    '''Function to ensure that the libraries used are consistent to avoid errors.'''
    try:
        pkg = pkg_resources.get_distribution(package)
        if pkg.version != version:
            raise pkg_resources.VersionConflict(pkg, version)
    except (pkg_resources.DistributionNotFound, pkg_resources.VersionConflict):
        subprocess.check_call(["pip", "install", f"{package}=={version}"])

install_if_needed("langchain-core", "0.3.72")
install_if_needed("langchain-openai", "0.3.28")
install_if_needed("langchain-community", "0.3.27")
install_if_needed("unstructured", "0.18.11")
install_if_needed("langchain-chroma", "0.2.5")
install_if_needed("langchain-text-splitters", "0.3.9")
install_if_needed("pydantic", "2.11.9")

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_core-0.3.72-py3-none-any.whl.metadata (5.8 kB)
Using cached langchain_core-0.3.72-py3-none-any.whl (442 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.81
    Uninstalling langchain-core-0.3.81:
      Successfully uninstalled langchain-core-0.3.81


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.30.11 requires langchain<0.2.0,>=0.1.10, but you have langchain 0.3.27 which is incompatible.
embedchain 0.1.113 requires chromadb<0.5.0,>=0.4.24, but you have chromadb 1.4.0 which is incompatible.
embedchain 0.1.113 requires langchain<0.2.0,>=0.1.4, but you have langchain 0.3.27 which is incompatible.
embedchain 0.1.113 requires langchain-openai<0.2.0,>=0.1.7, but you have langchain-openai 0.3.28 which is incompatible.
embedchain 0.1.113 requires pypdf<5.0.0,>=4.0.1, but you have pypdf 6.5.0 which is incompatible.
langchain-cohere 0.1.5 requires langchain-core<0.3,>=0.1.42, but you have langchain-core 0.3.72 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


...


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


In [16]:
# Import the required packages
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [17]:
# Load the HTML as a LangChain document loader
loader = UnstructuredHTMLLoader(file_path="data/mg-zs-warning-messages.html")
car_docs = loader.load()

In [18]:
import os

# Load the models required to complete the exercise
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.environ["OPENAI_API_KEY"])

In [19]:
# extract what's inside the document as an overview
print(car_docs[0].page_content)
print(car_docs[0].metadata)

Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.

Warning Message Procedure Low Oil Pressure Indicates that the oil pressure is too low, which may result in severe engine damage. As soon as safety permits, stop the car, switch off the engine and check the engine oil

In [20]:
len("Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault.")

109

In [21]:
# Step-2 Split the HTML Documents into Chunks Using RecursiveCharacterTextSplitter.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 120,
    chunk_overlap= 20,
)
docs = text_splitter.split_documents(car_docs)
print(docs[0].page_content)
print(docs[1].metadata)
print(len(docs))

Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please
{'source': 'data/mg-zs-warning-messages.html'}
63


In [22]:
# Step-3 Embed each Chunk and store it into ChromaDB Vector Database.
ChromaDB = Chroma.from_documents(
    documents= docs,
    embedding = embeddings
)
# Last Step Create Retriever to retrieve document from the Database
retriever = ChromaDB.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 3}
)

In [23]:
# Create Prompt.
Prompt = """
 Use the only the Context provided to answer the following Query. If you don't know the answer, reply that you are unsure.
Context: {Context}
Query: {Query}
"""

In [24]:
# Create PromptTemplate
PromptTemplate = ChatPromptTemplate.from_template(Prompt)
PromptTemplate

ChatPromptTemplate(input_variables=['Context', 'Query'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['Context', 'Query'], input_types={}, partial_variables={}, template="\n Use the only the Context provided to answer the following Query. If you don't know the answer, reply that you are unsure.\nContext: {Context}\nQuery: {Query}\n"), additional_kwargs={})])

In [25]:
# Create RAG Chain using LangChain
chain = ({"Context": retriever , "Query": RunnablePassthrough()}) | PromptTemplate | llm

In [26]:
# invoke Chain
answer = chain.invoke("The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?").content
answer

'The "Gasoline Particular Filter Full" warning indicates that the gasoline particular filter is full. You should seek a repairer as soon as possible to address this issue.'